# Q-Learning

In [1]:
import os
import sys

cwd = os.getcwd()

parentdir = os.path.dirname(cwd)
sys.path.insert(0, parentdir)

from environments.tic_tac_toe.tic_tac_toe import TicTacToe
from environments.game import PlayRoom

In [ ]:
playroom = PlayRoom(game = TicTacToe())
